# LangGraph MCP (Model Context Protocol) 튜토리얼

이 튜토리얼에서는 LangGraph와 MCP(Model Context Protocol)를 통합하여 강력한 AI 에이전트를 구축하는 방법을 배웁니다.

## 학습 목표
- MCP의 개념과 아키텍처 이해
- MultiServerMCPClient를 사용한 다중 서버 관리
- React Agent 및 ToolNode와 MCP 통합
- 실전 예제를 통한 복잡한 에이전트 구축

## 목차
1. **MCP 개요 및 설치**
2. **기본 MCP 서버 생성**
3. **MultiServerMCPClient 설정**
4. **React Agent와 MCP 통합**
5. **ToolNode와 MCP 통합**
6. **다중 MCP 서버 관리**
7. **실전 예제 - 복잡한 에이전트 구축**

## Part 1: MCP 개요 및 설치

### MCP(Model Context Protocol)란?

MCP는 애플리케이션이 언어 모델에 도구와 컨텍스트를 제공하는 방법을 표준화한 오픈 프로토콜입니다.

#### 주요 특징:
- 🔧 **표준화된 도구 인터페이스**: 일관된 방식으로 도구 정의 및 사용
- 🌐 **다양한 전송 메커니즘**: stdio, HTTP, WebSocket 등 지원
- 🔄 **동적 도구 검색**: 런타임에 도구 자동 검색 및 로드
- 🏗️ **확장 가능한 아키텍처**: 여러 서버를 동시에 연결 가능

### 설치

MCP를 사용하기 위해 필요한 패키지를 설치합니다:

In [1]:
# 필요한 라이브러리 임포트
import os
import nest_asyncio
from typing import Annotated, List, Dict, Any
from dataclasses import dataclass
from datetime import datetime

# LangChain & LangGraph
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

# MCP
from langchain_mcp_adapters.client import MultiServerMCPClient

# 환경 변수 설정
from dotenv import load_dotenv

load_dotenv()

print("✅ 패키지 임포트 완료!")

✅ 패키지 임포트 완료!


In [2]:
nest_asyncio.apply()

## Part 2: 기본 MCP 서버 생성

MCP 서버는 도구를 제공하는 독립적인 프로세스입니다. FastMCP를 사용하여 간단한 서버를 만들어봅시다.

### 예제: 수학 연산 MCP 서버

In [ ]:
# math_server.py 파일 생성
math_server_code = '''
#!/usr/bin/env python
"""Simple math operations MCP server."""

from mcp.server.fastmcp import FastMCP
from typing import Annotated

# MCP 서버 인스턴스 생성
mcp = FastMCP("Math Operations")

@mcp.tool()
def add(
    a: Annotated[float, "First number"],
    b: Annotated[float, "Second number"]
) -> float:
    """Add two numbers together."""
    return a + b

@mcp.tool()
def multiply(
    a: Annotated[float, "First number"],
    b: Annotated[float, "Second number"]
) -> float:
    """Multiply two numbers."""
    return a * b

@mcp.tool()
def divide(
    a: Annotated[float, "Numerator"],
    b: Annotated[float, "Denominator"]
) -> float:
    """Divide two numbers."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

@mcp.tool()
def power(
    base: Annotated[float, "Base number"],
    exponent: Annotated[float, "Exponent"]
) -> float:
    """Calculate base raised to the power of exponent."""
    return base ** exponent

if __name__ == "__main__":
    # stdio 전송 방식으로 서버 실행
    mcp.run(transport="stdio")
'''

# 파일 저장
with open("math_server.py", "w") as f:
    f.write(math_server_code)

print("✅ math_server.py 파일이 생성되었습니다!")

✅ math_server.py 파일이 생성되었습니다!


In [4]:
# 날씨 정보 서버 생성
weather_server_code = '''
#!/usr/bin/env python
"""Weather information MCP server."""

from mcp.server.fastmcp import FastMCP
from typing import Annotated
import random
from datetime import datetime

mcp = FastMCP("Weather Service")

@mcp.tool()
def get_weather(
    city: Annotated[str, "City name"]
) -> dict:
    """Get current weather for a city."""
    # 실제로는 API 호출을 하겠지만, 여기서는 모의 데이터 반환
    weather_conditions = ["Sunny", "Cloudy", "Rainy", "Snowy", "Windy"]
    
    return {
        "city": city,
        "temperature": random.randint(-10, 35),
        "condition": random.choice(weather_conditions),
        "humidity": random.randint(30, 90),
        "wind_speed": random.randint(0, 30),
        "timestamp": datetime.now().isoformat()
    }

@mcp.tool()
def get_forecast(
    city: Annotated[str, "City name"],
    days: Annotated[int, "Number of days (1-7)"] = 3
) -> list:
    """Get weather forecast for a city."""
    if days < 1 or days > 7:
        raise ValueError("Days must be between 1 and 7")
    
    forecast = []
    for i in range(days):
        forecast.append({
            "day": i + 1,
            "high": random.randint(10, 35),
            "low": random.randint(-5, 20),
            "condition": random.choice(["Sunny", "Cloudy", "Rainy"])
        })
    
    return {
        "city": city,
        "forecast": forecast
    }

if __name__ == "__main__":
    mcp.run(transport="stdio")
'''

with open("weather_server.py", "w") as f:
    f.write(weather_server_code)

print("✅ weather_server.py 파일이 생성되었습니다!")

✅ weather_server.py 파일이 생성되었습니다!


## Part 3: MultiServerMCPClient 설정

MultiServerMCPClient를 사용하면 여러 MCP 서버를 동시에 관리할 수 있습니다.

### 기본 설정 패턴

In [10]:
# MultiServerMCPClient 설정 예제
async def setup_mcp_client():
    """MCP 클라이언트를 설정하고 도구를 가져옵니다."""

    # 서버 구성 정의
    server_configs = {
        "math": {
            "command": "python",
            "args": ["math_server.py"],
            "transport": "stdio",
            "env": {},  # 환경 변수 (필요시)
        },
        "weather": {
            "command": "python",
            "args": ["weather_server.py"],
            "transport": "stdio",
        },
    }

    # MCP 클라이언트 생성
    client = MultiServerMCPClient(server_configs)

    # 도구 가져오기
    tools = await client.get_tools()

    print(f"✅ {len(tools)} 개의 도구가 로드되었습니다:")
    for tool in tools:
        print(f"  - {tool.name}: {tool.description}")

    return client, tools


# 클라이언트 테스트
await setup_mcp_client()  # Jupyter에서 실행

✅ 6 개의 도구가 로드되었습니다:
  - add: Add two numbers together.
  - multiply: Multiply two numbers.
  - divide: Divide two numbers.
  - power: Calculate base raised to the power of exponent.
  - get_weather: Get current weather for a city.
  - get_forecast: Get weather forecast for a city.


(<langchain_mcp_adapters.client.MultiServerMCPClient at 0x1589991f0>,
 [StructuredTool(name='add', description='Add two numbers together.', args_schema={'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'addArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x15aed3560>),
  StructuredTool(name='multiply', description='Multiply two numbers.', args_schema={'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'multiplyArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x15f9f5800>),
  StructuredTool(name='divide', description='Divide two numbers.', args_schema={'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'

### HTTP 전송 방식 사용

원격 서버나 HTTP 엔드포인트를 사용하는 경우:

In [11]:
# HTTP 기반 MCP 서버 설정 예제
http_server_config = {
    "remote_math": {
        "url": "http://localhost:8000/mcp",
        "transport": "streamable_http",
        "headers": {"Authorization": "Bearer YOUR_TOKEN"},  # 선택적: 인증 헤더 등
    },
    "remote_weather": {
        "url": "http://api.weather.example.com/mcp",
        "transport": "streamable_http",
    },
}

print("📡 HTTP 전송 방식 설정 예제:")
print(http_server_config)

📡 HTTP 전송 방식 설정 예제:
{'remote_math': {'url': 'http://localhost:8000/mcp', 'transport': 'streamable_http', 'headers': {'Authorization': 'Bearer YOUR_TOKEN'}}, 'remote_weather': {'url': 'http://api.weather.example.com/mcp', 'transport': 'streamable_http'}}


## Part 4: React Agent와 MCP 통합

React Agent는 추론(Reason)과 행동(Act)을 반복하는 패턴을 구현합니다. MCP 도구와 함께 사용하면 강력한 에이전트를 만들 수 있습니다.

In [12]:
async def create_mcp_react_agent():
    """MCP 도구를 사용하는 React Agent를 생성합니다."""

    # MCP 클라이언트 설정
    server_configs = {
        "math": {"command": "python", "args": ["math_server.py"], "transport": "stdio"},
        "weather": {
            "command": "python",
            "args": ["weather_server.py"],
            "transport": "stdio",
        },
    }

    # MCP 클라이언트 생성 및 도구 가져오기
    client = MultiServerMCPClient(server_configs)
    tools = await client.get_tools()

    # LLM 설정
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # React Agent 생성
    agent = create_react_agent(
        llm, tools, checkpointer=InMemorySaver()  # 상태 저장을 위한 체크포인터
    )

    return agent, client


# 에이전트 사용 예제
async def test_react_agent():
    """React Agent 테스트"""
    agent, client = await create_mcp_react_agent()

    # 대화 설정
    config = {"configurable": {"thread_id": "test-thread-1"}}

    # 질문 1: 수학 계산
    response = await agent.ainvoke(
        {"messages": [HumanMessage("25의 제곱은 얼마인가요?")]}, config
    )
    print("\n🤖 Agent Response 1:")
    print(response["messages"][-1].content)

    # 질문 2: 날씨 정보
    response = await agent.ainvoke(
        {"messages": [HumanMessage("서울의 날씨를 알려주세요.")]}, config
    )
    print("\n🤖 Agent Response 2:")
    print(response["messages"][-1].content)

    # 질문 3: 복합 질문
    response = await agent.ainvoke(
        {
            "messages": [
                HumanMessage(
                    "만약 서울의 온도가 섭씨라면, 화씨로 변환하면 얼마인가요? "
                    "(화씨 = 섭씨 * 9/5 + 32)"
                )
            ]
        },
        config,
    )
    print("\n🤖 Agent Response 3:")
    print(response["messages"][-1].content)

    # MultiServerMCPClient는 자동으로 정리됨
    print("\n✅ 테스트 완료!")


# 실행 (Jupyter에서)
await test_react_agent()


🤖 Agent Response 1:
25의 제곱은 625입니다.

🤖 Agent Response 2:
서울의 현재 날씨는 다음과 같습니다:

- **온도**: 2°C
- **날씨 상태**: 바람이 많이 불고 있습니다.
- **습도**: 68%
- **바람 속도**: 26 km/h

날씨에 유의하시기 바랍니다!

🤖 Agent Response 3:
서울의 온도 2°C는 화씨로 변환하면 35.6°F입니다.

✅ 테스트 완료!


## Part 5: ToolNode와 MCP 통합

ToolNode를 사용하면 더 세밀한 제어가 가능한 커스텀 워크플로우를 만들 수 있습니다.

In [13]:
# 상태 정의
@dataclass
class AgentState:
    """에이전트 상태"""

    messages: Annotated[List[BaseMessage], add_messages]
    context: Dict[str, Any] = None  # 추가 컨텍스트 정보


async def create_mcp_workflow():
    """MCP 도구를 사용하는 커스텀 워크플로우 생성"""

    # MCP 클라이언트 설정
    server_configs = {
        "math": {"command": "python", "args": ["math_server.py"], "transport": "stdio"},
        "weather": {
            "command": "python",
            "args": ["weather_server.py"],
            "transport": "stdio",
        },
    }

    client = MultiServerMCPClient(server_configs)
    tools = await client.get_tools()

    # LLM 설정 (도구 바인딩)
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    llm_with_tools = llm.bind_tools(tools)

    # 워크플로우 그래프 생성
    workflow = StateGraph(AgentState)

    # 노드 정의
    async def agent_node(state: AgentState):
        """LLM을 호출하여 응답 생성"""
        response = await llm_with_tools.ainvoke(state.messages)
        return {"messages": [response]}

    # ToolNode 생성
    tool_node = ToolNode(tools)

    # 조건부 라우팅 함수
    def should_continue(state: AgentState):
        """도구 호출이 필요한지 확인"""
        last_message = state.messages[-1]

        # 도구 호출이 있으면 tool_node로
        if hasattr(last_message, "tool_calls") and last_message.tool_calls:
            return "tools"
        # 없으면 종료
        return END

    # 그래프 구성
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", tool_node)

    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges(
        "agent", should_continue, {"tools": "tools", END: END}
    )
    workflow.add_edge("tools", "agent")

    # 컴파일
    app = workflow.compile(checkpointer=InMemorySaver())

    return app, client


# 워크플로우 테스트
async def test_workflow():
    """커스텀 워크플로우 테스트"""
    app, client = await create_mcp_workflow()

    config = {"configurable": {"thread_id": "workflow-test-1"}}

    # 스트리밍으로 실행
    async for event in app.astream(
        {
            "messages": [
                HumanMessage(
                    "서울의 날씨를 확인하고, 온도가 20도 이상이면 "
                    "20의 제곱을 계산해주세요."
                )
            ]
        },
        config,
    ):
        for key, value in event.items():
            if key == "agent":
                print(f"\n🤖 Agent: {value['messages'][-1].content}")
            elif key == "tools":
                print(f"\n🔧 Tool executed")

    # MultiServerMCPClient는 자동으로 정리됨
    print("\n✅ 워크플로우 테스트 완료!")


# 실행 (Jupyter에서)
await test_workflow()


🤖 Agent: 

🔧 Tool executed

🤖 Agent: 

🔧 Tool executed

🤖 Agent: 서울의 현재 온도는 4도입니다. 따라서 20도의 제곱을 계산한 결과는 400입니다.

✅ 워크플로우 테스트 완료!


## Part 6: 다중 MCP 서버 관리

여러 MCP 서버를 효율적으로 관리하는 고급 패턴을 살펴봅시다.

### 동적 서버 관리

In [14]:
class MCPServerManager:
    """MCP 서버를 동적으로 관리하는 클래스"""

    def __init__(self):
        self.server_configs = {}
        self.client = None
        self.tools = []

    def add_server(self, name: str, config: dict):
        """서버 구성 추가"""
        self.server_configs[name] = config
        print(f"✅ 서버 '{name}' 추가됨")

    def remove_server(self, name: str):
        """서버 구성 제거"""
        if name in self.server_configs:
            del self.server_configs[name]
            print(f"❌ 서버 '{name}' 제거됨")

    async def initialize(self):
        """클라이언트 초기화 및 도구 로드"""
        # 기존 클라이언트 정리
        self.client = None

        self.client = MultiServerMCPClient(self.server_configs)
        self.tools = await self.client.get_tools()

        print(f"\n🚀 초기화 완료!")
        print(f"📦 로드된 서버: {list(self.server_configs.keys())}")
        print(f"🔧 사용 가능한 도구: {len(self.tools)}개")

        return self.tools

    async def cleanup(self):
        """정리 작업"""
        # MultiServerMCPClient는 자동으로 정리됨
        self.client = None
        self.tools = []
        print("🧹 정리 완료")

    def get_tool_by_name(self, tool_name: str):
        """이름으로 도구 찾기"""
        for tool in self.tools:
            if tool.name == tool_name:
                return tool
        return None

    def list_tools(self):
        """도구 목록 출력"""
        print("\n📋 사용 가능한 도구:")
        for i, tool in enumerate(self.tools, 1):
            print(f"  {i}. {tool.name}: {tool.description}")


# 서버 매니저 사용 예제
async def demo_server_manager():
    """서버 매니저 데모"""
    manager = MCPServerManager()

    # 서버 추가
    manager.add_server(
        "math", {"command": "python", "args": ["math_server.py"], "transport": "stdio"}
    )

    manager.add_server(
        "weather",
        {"command": "python", "args": ["weather_server.py"], "transport": "stdio"},
    )

    # 초기화
    await manager.initialize()

    # 도구 목록 표시
    manager.list_tools()

    # 정리
    await manager.cleanup()


# 실행 (Jupyter에서)
await demo_server_manager()

✅ 서버 'math' 추가됨
✅ 서버 'weather' 추가됨

🚀 초기화 완료!
📦 로드된 서버: ['math', 'weather']
🔧 사용 가능한 도구: 6개

📋 사용 가능한 도구:
  1. add: Add two numbers together.
  2. multiply: Multiply two numbers.
  3. divide: Divide two numbers.
  4. power: Calculate base raised to the power of exponent.
  5. get_weather: Get current weather for a city.
  6. get_forecast: Get weather forecast for a city.
🧹 정리 완료


### 오류 처리 및 재시도 전략

In [15]:
import asyncio
from typing import Optional


class RobustMCPClient:
    """오류 처리와 재시도 기능을 갖춘 강건한 MCP 클라이언트"""

    def __init__(self, server_configs: dict, max_retries: int = 3):
        self.server_configs = server_configs
        self.max_retries = max_retries
        self.client = None
        self.failed_servers = set()

    async def connect_with_retry(self) -> Optional[MultiServerMCPClient]:
        """재시도 로직을 포함한 연결"""

        for attempt in range(self.max_retries):
            try:
                print(f"\n🔄 연결 시도 {attempt + 1}/{self.max_retries}...")

                # 실패한 서버 제외하고 연결 시도
                active_configs = {
                    name: config
                    for name, config in self.server_configs.items()
                    if name not in self.failed_servers
                }

                if not active_configs:
                    print("❌ 사용 가능한 서버가 없습니다.")
                    return None

                self.client = MultiServerMCPClient(active_configs)
                tools = await self.client.get_tools()

                print(f"✅ 연결 성공! {len(tools)}개 도구 로드됨")
                return self.client

            except Exception as e:
                print(f"⚠️ 연결 실패: {e}")

                if attempt < self.max_retries - 1:
                    wait_time = 2**attempt  # 지수 백오프
                    print(f"⏳ {wait_time}초 후 재시도...")
                    await asyncio.sleep(wait_time)
                else:
                    print("❌ 최대 재시도 횟수 초과")
                    return None

    async def call_tool_safe(self, tool_name: str, **kwargs):
        """안전한 도구 호출 (오류 처리 포함)"""
        try:
            if not self.client:
                await self.connect_with_retry()

            # 도구 찾기
            tools = await self.client.get_tools()
            tool = next((t for t in tools if t.name == tool_name), None)

            if not tool:
                raise ValueError(f"도구 '{tool_name}'을 찾을 수 없습니다.")

            # 도구 실행
            result = await tool.ainvoke(kwargs)
            return {"success": True, "result": result}

        except Exception as e:
            return {"success": False, "error": str(e)}

    async def health_check(self):
        """서버 상태 확인"""
        print("\n🏥 서버 상태 확인 중...")

        health_status = {}
        for server_name in self.server_configs:
            try:
                # 간단한 테스트 연결
                test_client = MultiServerMCPClient(
                    {server_name: self.server_configs[server_name]}
                )
                tools = await test_client.get_tools()
                # MultiServerMCPClient는 context manager를 지원하거나 별도 cleanup이 필요없을 수 있음

                health_status[server_name] = {
                    "status": "healthy",
                    "tools_count": len(tools),
                }
                print(f"  ✅ {server_name}: 정상 ({len(tools)} 도구)")

            except Exception as e:
                health_status[server_name] = {"status": "unhealthy", "error": str(e)}
                print(f"  ❌ {server_name}: 오류 - {e}")

        return health_status

    async def cleanup(self):
        """클라이언트 정리"""
        # MultiServerMCPClient는 별도의 cleanup이 필요없을 수 있음
        self.client = None
        self.failed_servers.clear()
        print("🧹 클라이언트 정리 완료")


# 사용 예제
async def test_robust_client():
    """강건한 클라이언트 테스트"""

    configs = {
        "math": {"command": "python", "args": ["math_server.py"], "transport": "stdio"},
        "weather": {
            "command": "python",
            "args": ["weather_server.py"],
            "transport": "stdio",
        },
    }

    client = RobustMCPClient(configs, max_retries=3)

    # 상태 확인
    await client.health_check()

    # 연결
    await client.connect_with_retry()

    # 안전한 도구 호출
    result = await client.call_tool_safe("add", a=5, b=3)
    print(f"\n📊 도구 호출 결과: {result}")

    # 정리
    await client.cleanup()


# 실행 (Jupyter에서)
await test_robust_client()


🏥 서버 상태 확인 중...
  ✅ math: 정상 (4 도구)
  ✅ weather: 정상 (2 도구)

🔄 연결 시도 1/3...
✅ 연결 성공! 6개 도구 로드됨

📊 도구 호출 결과: {'success': True, 'result': '8.0'}
🧹 클라이언트 정리 완료


## Part 7: 실전 예제 - 복잡한 에이전트 구축

모든 개념을 종합하여 실제 사용 가능한 복잡한 에이전트를 구축해봅시다.

### 예제: 데이터 분석 어시스턴트

In [17]:
# 데이터 분석 서버 생성
data_analysis_server = '''
#!/usr/bin/env python
"""Data analysis MCP server."""

from mcp.server.fastmcp import FastMCP
from typing import Annotated, List, Dict
import json
import statistics

mcp = FastMCP("Data Analysis")

@mcp.tool()
def calculate_statistics(
    data: Annotated[List[float], "List of numbers"]
) -> Dict:
    """Calculate basic statistics for a dataset."""
    if not data:
        return {"error": "Empty dataset"}
    
    return {
        "count": len(data),
        "mean": statistics.mean(data),
        "median": statistics.median(data),
        "stdev": statistics.stdev(data) if len(data) > 1 else 0,
        "min": min(data),
        "max": max(data)
    }

@mcp.tool()
def detect_outliers(
    data: Annotated[List[float], "List of numbers"],
    threshold: Annotated[float, "Z-score threshold"] = 2.0
) -> Dict:
    """Detect outliers using z-score method."""
    if len(data) < 3:
        return {"error": "Need at least 3 data points"}
    
    mean = statistics.mean(data)
    stdev = statistics.stdev(data)
    
    outliers = []
    for i, value in enumerate(data):
        z_score = abs((value - mean) / stdev) if stdev > 0 else 0
        if z_score > threshold:
            outliers.append({"index": i, "value": value, "z_score": z_score})
    
    return {
        "outliers": outliers,
        "count": len(outliers),
        "percentage": len(outliers) / len(data) * 100
    }

@mcp.tool()
def generate_report(
    title: Annotated[str, "Report title"],
    data: Annotated[Dict, "Data to include in report"]
) -> str:
    """Generate a formatted analysis report."""
    report = f"# {title}\\n\\n"
    report += f"Generated at: {datetime.now().isoformat()}\\n\\n"
    
    for section, content in data.items():
        report += f"## {section}\\n"
        if isinstance(content, dict):
            for key, value in content.items():
                report += f"- {key}: {value}\\n"
        else:
            report += f"{content}\\n"
        report += "\\n"
    
    return report

if __name__ == "__main__":
    mcp.run(transport="stdio")
'''

with open("data_analysis_server.py", "w") as f:
    f.write(data_analysis_server)

print("✅ data_analysis_server.py 파일이 생성되었습니다!")

✅ data_analysis_server.py 파일이 생성되었습니다!


In [20]:
# 복잡한 데이터 분석 에이전트
class DataAnalysisAgent:
    """MCP를 활용한 데이터 분석 에이전트"""

    def __init__(self):
        self.client = None
        self.agent = None
        self.analysis_history = []

    async def initialize(self):
        """에이전트 초기화"""
        # MCP 서버 구성
        server_configs = {
            "math": {
                "command": "python",
                "args": ["math_server.py"],
                "transport": "stdio",
            },
            "weather": {
                "command": "python",
                "args": ["weather_server.py"],
                "transport": "stdio",
            },
            "data_analysis": {
                "command": "python",
                "args": ["data_analysis_server.py"],
                "transport": "stdio",
            },
        }

        # MCP 클라이언트 생성
        self.client = MultiServerMCPClient(server_configs)
        tools = await self.client.get_tools()

        # LLM 설정
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

        # 커스텀 시스템 프롬프트
        system_prompt = """
        You are a professional data analyst assistant. Your role is to:
        1. Analyze data using available tools
        2. Identify patterns and outliers
        3. Provide actionable insights
        4. Generate comprehensive reports
        
        Always be thorough and explain your analysis clearly.
        """

        # React Agent 생성
        self.agent = create_react_agent(
            llm, tools, prompt=system_prompt, checkpointer=InMemorySaver()
        )

        print("✅ 데이터 분석 에이전트 초기화 완료")
        print(f"🔧 사용 가능한 도구: {len(tools)}개")

    async def analyze_dataset(self, data: List[float], context: str = ""):
        """데이터셋 분석"""
        config = {
            "configurable": {"thread_id": f"analysis-{len(self.analysis_history)}"}
        }

        # 분석 요청 구성
        request = f"""
        다음 데이터를 분석해주세요:
        데이터: {data}
        
        분석 요구사항:
        1. 기본 통계량 계산
        2. 이상치 탐지 (z-score 2.0 기준)
        3. 인사이트 도출
        4. 분석 보고서 생성
        
        컨텍스트: {context if context else "없음"}
        """

        # 에이전트 실행
        response = await self.agent.ainvoke(
            {"messages": [HumanMessage(request)]}, config
        )

        # 결과 저장
        analysis_result = {
            "timestamp": datetime.now().isoformat(),
            "data": data,
            "context": context,
            "analysis": response["messages"][-1].content,
        }
        self.analysis_history.append(analysis_result)

        return analysis_result

    async def compare_datasets(
        self,
        dataset1: List[float],
        dataset2: List[float],
        labels: tuple = ("Dataset 1", "Dataset 2"),
    ):
        """두 데이터셋 비교 분석"""
        config = {
            "configurable": {"thread_id": f"comparison-{len(self.analysis_history)}"}
        }

        request = f"""
        두 데이터셋을 비교 분석해주세요:
        
        {labels[0]}: {dataset1}
        {labels[1]}: {dataset2}
        
        다음 항목들을 비교해주세요:
        1. 평균과 중앙값의 차이
        2. 분산도 비교
        3. 이상치 비율
        4. 주요 차이점과 유사점
        5. 비즈니스 인사이트
        """

        response = await self.agent.ainvoke(
            {"messages": [HumanMessage(request)]}, config
        )

        return response["messages"][-1].content

    async def forecast_trend(self, historical_data: List[float], periods: int = 3):
        """트렌드 예측"""
        config = {
            "configurable": {"thread_id": f"forecast-{len(self.analysis_history)}"}
        }

        request = f"""
        과거 데이터를 기반으로 향후 {periods}개 기간의 트렌드를 예측해주세요:
        
        과거 데이터: {historical_data}
        
        다음을 포함해주세요:
        1. 트렌드 방향 (상승/하락/횡보)
        2. 예상 값 범위
        3. 신뢰도
        4. 주의사항
        """

        response = await self.agent.ainvoke(
            {"messages": [HumanMessage(request)]}, config
        )

        return response["messages"][-1].content

    async def cleanup(self):
        """정리 작업"""
        # MultiServerMCPClient는 자동으로 정리됨
        self.client = None
        self.agent = None
        print("🧹 에이전트 정리 완료")


# 사용 예제
async def demo_data_analysis():
    """데이터 분석 에이전트 데모"""

    agent = DataAnalysisAgent()
    await agent.initialize()

    try:
        # 예제 1: 단일 데이터셋 분석
        print("\n📊 예제 1: 판매 데이터 분석")
        sales_data = [100, 120, 115, 130, 125, 140, 135, 150, 145, 160, 300, 155]
        result = await agent.analyze_dataset(
            sales_data, context="월별 판매량 (단위: 천 개)"
        )
        print(result["analysis"])

        # 예제 2: 데이터셋 비교
        print("\n📊 예제 2: 두 지점 판매 비교")
        branch_a = [100, 110, 120, 115, 125, 130]
        branch_b = [95, 105, 115, 120, 130, 140]
        comparison = await agent.compare_datasets(
            branch_a, branch_b, labels=("지점 A", "지점 B")
        )
        print(comparison)

        # 예제 3: 트렌드 예측
        print("\n📊 예제 3: 향후 트렌드 예측")
        historical = [100, 105, 103, 108, 112, 115, 118, 122]
        forecast = await agent.forecast_trend(historical, periods=3)
        print(forecast)

    finally:
        await agent.cleanup()


# 실행 (Jupyter에서)
await demo_data_analysis()

✅ 데이터 분석 에이전트 초기화 완료
🔧 사용 가능한 도구: 9개

📊 예제 1: 판매 데이터 분석
### 데이터 분석 결과

#### 1. 기본 통계량
- **데이터 개수 (Count)**: 12
- **평균 (Mean)**: 147.92 천 개
- **중앙값 (Median)**: 137.5 천 개
- **표준편차 (Standard Deviation)**: 50.97 천 개
- **최소값 (Min)**: 100.0 천 개
- **최대값 (Max)**: 300.0 천 개

#### 2. 이상치 탐지
- **이상치 개수**: 1
- **이상치 비율**: 8.33%
- **이상치 값**: 300.0 천 개
- **이상치의 z-score**: 2.98

#### 3. 인사이트 도출
- 평균 판매량은 147.92 천 개로, 중앙값 137.5 천 개보다 높습니다. 이는 데이터의 분포가 오른쪽으로 치우쳐 있음을 나타냅니다.
- 표준편차가 50.97 천 개로 상당히 크기 때문에, 판매량의 변동성이 큽니다.
- 300.0 천 개의 판매량은 다른 데이터 포인트에 비해 매우 높은 값으로, 이는 이상치로 간주됩니다. 이 값은 전체 데이터의 평균과 중앙값에 큰 영향을 미칠 수 있습니다.

#### 4. 분석 보고서 생성
보고서 생성에 문제가 발생했습니다. 보고서 내용을 수동으로 작성하겠습니다.

---

### 월별 판매량 분석 보고서

**분석 개요**: 본 보고서는 월별 판매량 데이터를 분석하여 기본 통계량, 이상치 탐지 및 인사이트를 도출하였습니다.

**기본 통계량**:
- 데이터 개수: 12
- 평균: 147.92 천 개
- 중앙값: 137.5 천 개
- 표준편차: 50.97 천 개
- 최소값: 100.0 천 개
- 최대값: 300.0 천 개

**이상치 탐지**:
- 이상치 개수: 1 (300.0 천 개)
- 이상치 비율: 8.33%

**인사이트**:
- 판매량의 평균과 중앙값의 차이는 데이터의 비대칭성을 나타내며, 이상치가 전체 데이터에 미치는 영향을 고려해야 합니다